In [8]:
#libraries

import pandas as pd
import datetime
import psycopg2
import numpy as np
import os
# import seaborn as sns
# pd.set_option("display.max_columns",999)
# import seaborn as sns

# import warnings
# warnings.filterwarnings('ignore')

# pd.options.display.float_format = '{:.2f}'.format
# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.style.use('seaborn-white')

In [2]:
os.getcwd()

'C:\\Users\\clark\\Documents\\LOCAL_REPOSITORY_LOCATION\\DAPT\\costar\\downtime_viz'

In [5]:
# file paths

LOCAL = os.environ['LOCAL_REPOSITORY_LOCATION']
fdr = 'DAPT\costar\downtime_viz'
fn = 'rent_submarket.xlsx'
path_file = os.path.join(LOCAL, fdr, fn)


In [9]:
# RDS Connection

conn = psycopg2.connect(
    host = 'lease-data.cnzawwknyviz.us-east-1.rds.amazonaws.com',
    port = 5432,
    user = 'costar',
    password = 'Costar12',
    database='costar'
    )
cursor = conn.cursor() 

In [45]:
def read_data_merge():

# read downtime table from RDS

    q1 = '''SELECT * FROM downtime_lease_nov8'''

    downtime = pd.read_sql(q1, 
                           con = conn)

    # read rent table from excel or RDS (try RDS first)
    try:
        # read RDW

        q1 = '''SELECT * FROM rent_submarket'''

        rent = pd.read_sql(q1, 
                           con = conn)
    except:
        # read excel
        rent = pd.read_excel(path_file)

    rent.rename(columns = {'SubmarketName': 'submarket_name', 
                           'CBSAID': 'cbsaid',
                           'BuildingRatingID': 'building_rating_id'}, inplace = True)

    downtime.cbsaid = pd.to_numeric(downtime.cbsaid)

    df = pd.merge(downtime, 
                  rent, 
                  how = 'left', 
                  on = ['cbsaid', 'submarket_name', 'building_rating_id'])

    df.to_csv(os.path.join(LOCAL, fdr, 'downtime_rent.csv'), index = False)
    
read_data_merge()